In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit    
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

print('Starting')
# parameters
customer_id = '5'
formatted_attribute = 'width'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
dfs=dfs[0:4000]

custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

reg_ex(df)
print('Length of Total Dataframe: '+str(len(df)))
df.drop_duplicates(subset='external_id', inplace=True)
df=df[(df['external_id'].astype(str)!='200075904')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')&(df['external_id'].astype(str)!='200046141')&(df['external_id'].astype(str)!='200046271')]
import collections
print('REPEATING External IDs: '+str([item for item, count in collections.Counter(df['external_id']).items() if count > 1]))

power=r'(?i)(power(?:[A-z]|\s){0,15}\d+\s?W|\d+\s?W(?:[A-z]|\s){0,10}power)|()'
df['power'] = df['long_desc'].apply(lambda x: re_extract(power, str(x)))
power=df[df['power']!='[]']
print(len(power))
notpower=power[power['power'].astype(str)=='[]']
print(len(notpower))

import time
today = time.strftime("%Y_%m_%d")
print(today)

Starting
Length of Total Dataframe: 4000
REPEATING External IDs: []
4000
3989
2023_04_25


In [2]:
trips='''(?i)(\d+(?:.?\d+\/\d+|\/\d+|\.\d+)?.{0,5}x.{0,5}\d+(?:.?\d+\/\d+|\/\d+|\.\d+)?.{0,5}x.{0,5}\d+(?:.?\d+\/\d+|\/\d+|\.\d+)?.{0,5}(?:L\s?x\s?W\s?x\s?H|W\s?x\s?H\s?x\s?L|L\s?x\s?H\s?x\s?W|W\s?x\s?L\s?x\s?H))|((?:L\s?x\s?W\s?x\s?H|W\s?x\s?H\s?x\s?L|L\s?x\s?H\s?x\s?W|W\s?x\s?L\s?x\s?H).{0,5}\d+(?:.?\d+\/\d+|\/\d+|\.\d+)?.{0,5}x.{0,5}\d+(?:.?\d+\/\d+|\/\d+|\.\d+)?.{0,5}x.{0,5}\d+(?:.?\d+\/\d+|\/\d+|\.\d+)?.{0,})'''             
df['units']=df['long_desc'].apply(lambda x: re_extract(trips, str(x)))





In [3]:
# x=df[(df['units'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]

y=df[df['blank_matches_long'].astype(str)!='[]']
print(len(y))
y=df[df['units'].astype(str)!='[]']
print(len(y))

2774
270


# Width

In [4]:
df_width= df[(df['width'].astype(str)!='[]')]
df_width['width']=df_width['width'].apply(lambda x: natsorted(x))
df_width['width_one']=df_width['width'].apply(lambda x: re.sub(r"",'',str(x)))
clean_data(df_width,'width_one')
print(len(df_width))
df_width['width_three']=df_width['three'].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]').apply(lambda x: re.sub(r'''((?<=\[)')|(\s?"\s?)|('(?=\]))|((?<=\,)')|('(?=\,))''','"',str(x))).apply(lambda x: re.sub(r'(?i)(inx)','in',str(x)))
df_width['width_four']=df_width['width_three'].apply(lambda x: re.sub(r'''((?<=\[)"\,)|((?<=\[)'\,)''','',str(x)))
rounding_inch_feet(df_width,'width_three','a-eghj-mo-su-z')
# df_width=df_width[df_width['rounding'].astype(str)=='[]']
# print(len(df_width))
curate(df_width, 'width_four', formatted_attribute)
print(len(df_width))
rounding_inch_feet(df_width, curation_col,'a-eghj-mo-su-z')
df_width=df_width[df_width['rounding'].astype(str)=='[]']
print(len(df_width))
matchwidth=df_width[['external_id',curation_col]]

44
Repeated External IDs: []
44
43


In [5]:
df_width[5000:5500]

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, LONG_DESCRIPTION, length, height, width, diameter, blank_matches, blank_matches_long, tripple_name, tripple_long, double_name, double_long, mm_name, mm_long, units, power, width_one, input, one, two, three, width_three, width_four, rounding, ones, twos, threes, fours, fives, Q:width]
Index: []

# Diameter

In [6]:
df_diam=df[(df['width'].astype(str)=='[]')&(df['diameter'].astype(str)!='[]')]
df_diam['height']=df_diam['diameter'].apply(lambda x: natsorted(x))
df_diam['height_final']=df_diam['height'].apply(lambda x: re.sub(r'''(?i)(\\\"\s?diameter)|(\\?(?:\"\”|inc?h?)?(?<!').(?:diameter|dia))|(\\?('')\W?(diameter|dia))|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)(\\?(?:(?<!')'(?!')|fo?o?e?e?t)\W?dia(?:meter))|(square\W?foot)|(\'\\xa.{0,2}diameter)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x))).apply(lambda x: re.sub(r'''\[|\]|'|\\|"''','',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

curate(df_diam, 'height_final', formatted_attribute)
df_diam[curation_col]=df_diam[curation_col].apply(lambda x: re.sub(r'mter','',str(x)))

print(len(df_diam))
rounding_inch_feet(df_diam, curation_col,'a-eghj-mo-su-z')
df_diam=df_diam[df_diam['rounding'].astype(str)=='[]']
print(len(df_diam))

matches_diam=df_diam[['external_id',curation_col]]
print(len(matches_diam))
rounding_inch_feet(df_diam, curation_col,'a-eghj-mo-su-z')

Repeated External IDs: []
33
1
1


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, LONG_DESCRIPTION, length, height, width, diameter, blank_matches, blank_matches_long, tripple_name, tripple_long, double_name, double_long, mm_name, mm_long, units, power, height_final, ones, twos, threes, fours, fives, Q:width, rounding]
Index: []

# More

In [7]:
df_more=df[(df['width'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['width'].astype(str)!='[]')]
print(len(df_more))
# df_more

0


In [8]:
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [9]:
df_more['matches_more']=df_more['width'].apply(lambda x: re.sub(r'''\\" W|".?W|''.?W|”.?W|˝.?W|”.?W|” w|″ W''',' in',str(x))).apply(lambda x: re.sub(r'''\\' W|'.?W|'.?W''',' ft',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_more['matches_more']=df_more['matches_more'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

In [10]:
df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x))).apply(lambda x: re.sub('ft in','ft',str(x)))                                                                           
df_more['matches_more']=df_more['matches_more'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
df_more['matches_more']=df_more['matches_more'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

df_more[curation_col]=df_more['matches_more']
matchesmore=df_more[['external_id',curation_col]]
print(len(matchesmore))

0


In [11]:
df_na=df[(df['width'].astype(str)=='[]')&(df['length'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['width'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]#&(df['new_matches'].astype(str)!='[]')]
df_na[curation_col]='n/a'
matchesnadf=df_na[['external_id',curation_col]]
print(len(matchesnadf)) 

1096


# Two Min Max (Min is width)

In [12]:
dubs=df[(df['double_long'].astype(str)!='[]')&(df['double_name'].astype(str)=='[]')]
print(len(dubs))

pat_split=r'''(?i)((?:(?<!\d\W)\d+\/\d+|\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?)\\?(?:"|”|:'{0,2}|\Winc?h?e?s?|\Wft|\Wf(?:oo|ee)?t)?.?x.?(?:(?<!\d\W)\d+\/\d+|\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?)\\?(?:"|”|:'{0,2}|\Winc?h?e?s?|\Wft|\Wf(?:oo|ee)?t)?)|()'''
df['matches_split'] = df['product_name'].apply(lambda x: re_extract(pat_split, str(x))).apply(lambda x: re.sub(r'''(?i)((?<=\d)\-F\w*)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)((?<=\d)\-I\w*)''',' in',str(x)))
dubs=df[(df['matches_split'].astype(str)!='[]')&(df['length'].astype(str)=='[]')&(df['width'].astype(str)=='[]')]
print(len(dubs))

1938
236


In [13]:
# dubs['match']=dubs['dub'].apply(lambda x: f'''["{x}"]''').apply(lambda x: re.sub(r'''(?i)((?<=')\d[^\,']*x\s?)''','',str(x))).apply(lambda x: re.sub(r'''(?i)(\["')''','["',str(x))).apply(lambda x: re.sub(r'''(?i)('"\]')''','"]',str(x)))
# dubs['match']=dubs['match'].apply(lambda x: re.sub(r'''(?i)(\,.{0,5}x)''',',"',str(x))).apply(lambda x: re.sub(r'''((?<=\[").{0,5}x)''','',str(x)))          
# dubs['match'].explode().value_counts()

In [14]:
# dubs=dubs[dubs['double_long'].astype(str)!='[]']
# dubs['dub']=dubs['double_long'].apply(lambda x: re.sub(r'''(?i)(\\{1,3})|(\-in)|(\s?\\{0,3}\s?"\s?|(?<!\d)\s(?!\d))|([a-wyz])|(”)|(\[)|(\])|(\\{0,2}xa?0(?!\.))|(®)|(\))''','',str(x))).apply(lambda x: re.sub(r" '",'',str(x))).apply(lambda x: re.sub(r"''",'"',str(x))).apply(lambda x: re.sub(r'(?i)\s?x\s?','x',str(x)))#.apply(lambda x: f'["{x}"]').astype(str)                   
# dubs=dubs[dubs['dub'].astype(str)!='[]']
# dubs['dub']=dubs['dub'].apply(lambda x: re.sub(r'''(?i)((?<=\d)\-(?=x))|((?<=\d)\-(?='\]))''','',str(x)))
# dubs['dub']=dubs['dub'].apply(lambda x: re.sub(r"(?<=\d)(?!\w)(?!')(?!\.)(?!\d)(?!\/)(?!\-)","'",str(x)))

# df1 = dubs['dub'].str[2:-2].str.split(',').explode().str.split(r'(?i)x', expand=True).fillna('')
# # df1[1]=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+\/d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))
# # df1[0]=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))

# x=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()
# y=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()

# dubs[['Max','Min']] = pd.concat([df1.astype(float).max(1).groupby(level=0).apply(list),df1.astype(float).min(1).groupby(level=0).apply(list)], axis=1)

# clean_data(dubs,'Min')
# dubs['doubs']=dubs['two'].apply(lambda x: re.sub(r'\[\s?\"\s?\[\s?(:"|")?','',str(x))).apply(lambda x: re.sub(r'"?\]"\]','"]',str(x)))
# curate(dubs,'doubs','width')
# rounding_inch_feet(dubs,curation_col,'a-eghj-mo-su-z')
# dubs=dubs[dubs['rounding'].astype(str)!='[]']

In [15]:
# dubs

In [16]:
# dubs['dub'][0:10]

In [17]:
# print(len(dubs))
# dubs['dub'][0:500]#.explode().value_counts()[0:500]
# df1[1][0:500]

In [18]:
# dubs['dub'].explode().value_counts()
# # dubs[dubs['dub'].astype(str)=='''['18x8-','18x8']''']
# df1[df1[0].astype(str)=='']
# dubs[0:500]

In [19]:
# match_dub=dubs[['external_id',curation_col]]

In [20]:
# pat_split=r'''(?i)(\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?e?s?)?(?:\Wft)?(?:\Wfoot)?).?x.?\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?e?s?)?(?:\Wft)?(?:\Wfoot)?))|()'''
regex_pattern_blank=r'(?i)(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
# pat_comma=r"\,"

# df['matches_split'] = df['product_name'].apply(lambda x: re_extract(pat_split, str(x)))
# df['comma']=df['matches_split'].apply(lambda x: re_extract(pat_comma, str(x)))
df['matchez'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

# split=df[(df['matches_split'].astype(str)!='[]')&(df['length'].astype(str)=='[]')&(df['width'].astype(str)=='[]')]
# split=split[split['comma'].astype(str)=='[]']
# print(len(split))

In [21]:
# split['matches_split']=split['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
# split['matches_split']=split['matches_split'].str.get(0).apply(lambda x: x.lower())
# split=split[split['matches_split'].astype(str)!="['13 x 13 inches', '6.5 x 6.5 inches']"]
# split[['length', 'widths']]=split['matches_split'].str.split("x",expand=True,)
# print(len(split))

In [22]:
# split['length']=split['length'].apply(lambda x: re.sub(r"\[\'",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|8x/1x|4x4 b|26x31 g|8x/1",'',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"2 24 in",'24 in',str(x))).apply(lambda x: re.sub(r"33 (?!in)",'33 in',str(x))).apply(lambda x: re.sub(r" ",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"5 inch",'31.5 in',str(x))).apply(lambda x: re.sub(r"20x24','20",'20 in',str(x)))                        
# split['widths']=split['widths'].apply(lambda x: re.sub(r"\'\]",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|None| |4-f",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"24 in','20x24 in",'24 in',str(x))).apply(lambda x: re.sub(r"27b",'27 in',str(x))).apply(lambda x: re.sub(r"24(?! )",'24 in',str(x))).apply(lambda x: re.sub(r'\[|\]','',str(x)))

# pat_split=r'''(?i)(ft)|()'''
# split['ft'] = split['widths'].apply(lambda x: re_extract(pat_split, str(x)))
# split['ft_length'] = split['length'].apply(lambda x: re_extract(pat_split, str(x)))
# print(len(split))

In [23]:
# split=split[split['length'].astype(str)!=""]
# split=split[split['widths'].astype(str)!=""]
# split=split[split['comma'].astype(str)=='[]']
# split_inch=split[(split['ft'].astype(str)=='[]')&(split['ft_length'].astype(str)=='[]')]

# split_inch['length']=split_inch['length'].apply(lambda x:re.sub(r'''\s?in|\s?inche?s?|ch|\(|\,|es|\['|'\]|\-|\?|\/''','',str(x)))
# split_inch['widths']=split_inch['widths'].apply(lambda x:re.sub(r'''\s?in|\s?inche?s?|ch|\(|\,|es|\['|'\]|\-|\?|\/''','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
# split_inch['length']=split_inch['length'].astype(float)
# split_inch['widths']=split_inch['widths'].astype(float)
# split_inch['Max'] = split_inch[['length', 'widths']].values.max(1)
# split_inch['Min'] = split_inch[['length', 'widths']].values.min(1)
# split_inch['Min']=split_inch['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# print(len(split_inch))
# split_inch[curation_col]=split_inch['Min']
# split_inch[curation_col]=split_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))
# rounding_inch_feet(split_inch, curation_col,'a-eghj-mo-su-z')

In [24]:
# msplit_inch=split_inch[['external_id',curation_col]]

# N/A values

In [25]:
regex_pattern_blank=r'(?i)(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['matchez'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [26]:
print(len(df[(df['matchez'].astype(str)=='[]')&(df['width'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['double_long'].astype(str)=='[]')&(df['double_name'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]))                                                                                         
na_values=df[(df['matchez'].astype(str)=='[]')&(df['width'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['double_long'].astype(str)=='[]')&(df['double_name'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]
na_values[curation_col]='n/a'
matches_na_values=na_values[['external_id',curation_col]]

1098


In [27]:
# df_name=df[['external_id','product_name','long_desc','height','blank_matches','blank_matches_long','tripple_name','tripple_long','double_name','na_matches']]
# regex_pattern_blank=r'(?i)(\d+(?:.Inch)\s?x\s?\d+(?:.Inch))|()'
# df_name['new_blank_matches'] = df_name['blank_matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_names=df_name[(df_name['new_blank_matches'].astype(str)!='[]')]
# print(len(df_names))

# regex_pattern_blank=r"(?i)(\d+'\s?x\s?\d+')|()"
# df_name['new_blank_matches_feet'] = df_name['blank_matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_names_ft=df_name[(df_name['new_blank_matches_feet'].astype(str)!='[]')]
# print(len(df_names_ft))

In [28]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnadf))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matches_na_values))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matches_diam))

matchesnadf
matches_na_values
matches_diam


In [29]:
print(len(df))
print(len(matchesnadf)+len(matches_na_values)+len(matches_diam)+len(matchwidth))#+len(msplit_inch)) #len(msplit_ft)+

4000
2238


In [30]:
# print(len(df[(df['width'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['width'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['double_long'].astype(str)=='[]')]))
# df[(df['width'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['double_long'].astype(str)=='[]')]

In [31]:
# rounding_inch_feet(split_inch,curation_col,'a-eghj-mo-su-z')

In [32]:
rounding_inch_feet(df_diam, curation_col,'a-eghj-mo-su-z')

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, LONG_DESCRIPTION, length, height, width, diameter, blank_matches, blank_matches_long, tripple_name, tripple_long, double_name, double_long, mm_name, mm_long, units, power, height_final, ones, twos, threes, fours, fives, Q:width, rounding]
Index: []

In [33]:
print(len(df))
a=df[(df['width'].astype(str)=='[]')&(df['matchez'].astype(str)!='[]')]
print(len(a))
df[df['units'].astype(str)=='[]'][0:500]

4000
468


buckets external_id  \
0                              Shelf Dividers/Organizers   201187101   
1                                                 Fryers   201179006   
6                                                 Fryers   201179021   
10                           Sugar Bowls/Jars/Dispensers    13404860   
11                     Flush & Semi-Flush Mount Lighting   201152586   
16                 Play Pens, Safety Gates, & Enclosures   200628691   
18                 Play Pens, Safety Gates, & Enclosures   200739075   
19                 Play Pens, Safety Gates, & Enclosures   200909848   
21                 Play Pens, Safety Gates, & Enclosures   200882944   
23                 Play Pens, Safety Gates, & Enclosures   200901669   
24                 Play Pens, Safety Gates, & Enclosures   200901670   
25                 Play Pens, Safety Gates, & Enclosures   200901671   
26                 Play Pens, Safety Gates, & Enclosures   200901672   
27                 Play Pens, Safety Gates, & Enclosures   200901673   
28                 Play Pens, Safety Gates, & Enclosures   200901783   
29                 Play Pens, Safety Gates, & Enclosures   200901795   
30                 Play Pens, Safety Gates, & Enclosures   200901797   
31                 Play Pens, Safety Gates, & Enclosures   200901798   
32                 Play Pens, Safety Gates, & Enclosures   200903835   
33                 Play Pens, Safety Gates, & Enclosures   200909847   
39                 Play Pens, Safety Gates, & Enclosures   200861251   
41                 Play Pens, Safety Gates, & Enclosures   200751831   
42                 Play Pens, Safety Gates, & Enclosures   200797713   
49                 Play Pens, Safety Gates, & Enclosures   200791021   
50                 Play Pens, Safety Gates, & Enclosures   200791011   
51                 Play Pens, Safety Gates, & Enclosures   200793361   
52                 Play Pens, Safety Gates, & Enclosures   200791013   
69                 Play Pens, Safety Gates, & Enclosures   200791014   
70                 Play Pens, Safety Gates, & Enclosures   200791020   
73                 Play Pens, Safety Gates, & Enclosures   200791026   
76                 Play Pens, Safety Gates, & Enclosures   200791050   
78                 Play Pens, Safety Gates, & Enclosures   200778577   
81                 Play Pens, Safety Gates, & Enclosures   200721120   
82                 Play Pens, Safety Gates, & Enclosures   200721123   
90                 Play Pens, Safety Gates, & Enclosures   200721210   
91                 Play Pens, Safety Gates, & Enclosures   200721209   
98                 Play Pens, Safety Gates, & Enclosures   200721137   
99                 Play Pens, Safety Gates, & Enclosures   200722447   
100                Play Pens, Safety Gates, & Enclosures   200782297   
101                Play Pens, Safety Gates, & Enclosures   200996007   
102                Play Pens, Safety Gates, & Enclosures   200996006   
103                Play Pens, Safety Gates, & Enclosures    66476023   
106                Play Pens, Safety Gates, & Enclosures   201044180   
107                Play Pens, Safety Gates, & Enclosures   201044181   
117                Play Pens, Safety Gates, & Enclosures   201044185   
151                Play Pens, Safety Gates, & Enclosures   201178922   
172                Play Pens, Safety Gates, & Enclosures   201126805   
173                Play Pens, Safety Gates, & Enclosures   201143309   
174                Play Pens, Safety Gates, & Enclosures   201143312   
175                Play Pens, Safety Gates, & Enclosures   201143313   
176                Play Pens, Safety Gates, & Enclosures   201143314   
183                Play Pens, Safety Gates, & Enclosures   201152708   
184                Play Pens, Safety Gates, & Enclosures   201152709   
186                Play Pens, Safety Gates, & Enclosures   200720636   
205                Play Pens, Safety Gates, & Enclosures   200945998   
206              

In [34]:
stop   

NameError: name 'stop' is not defined

# send to the folder for upload

In [35]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, matchesnadf,today)

In [36]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, match_dub,today)

NameError: name 'match_dub' is not defined

In [37]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, matches_na_values,today)

In [38]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matches_diam,today)

In [39]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matchwidth,today)